In [ ]:
#
# CONFIGURE RUN-TIME PARAMETERS FOR THIS NOTEBOOK
#
SERVER = " [ YOUR ID HERE ] .datawarehouse.fabric.microsoft.com"
DATABASE = "AdventureWorks_Warehouse"
DB_SCHEMA = "dbo"
CLIENT_ID = " [ YOUR ID HERE ] "
CLIENT_SECRET = " [ YOUR ID HERE ] "

print("✅ Successfully configured all paramaters for this run.")

✅ Successfully configured all paramaters for this run.


In [37]:
#
# Import Packages Required for this Notebook
#
import pyodbc
import textwrap

print("✅ Successfully imported all packages for this notebook.")
print(f"PyOdbc drivers available: {pyodbc.drivers()}")

✅ Successfully imported all packages for this notebook.
PyOdbc drivers available: ['ODBC Driver 18 for SQL Server']


In [34]:
#
# Create the SQL connection to the Fabric Warehouse
#

conn_str = f"""
DRIVER={{ODBC Driver 18 for SQL Server}};
SERVER={SERVER};
DATABASE={DATABASE};
Initial Catalog={DB_SCHEMA};
Authentication=ActiveDirectoryServicePrincipal;
UID={CLIENT_ID};
PWD={CLIENT_SECRET};
"""

# Create the connection
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()
cursor.execute("SELECT @@SPID")
spid = cursor.fetchone()[0]
cursor.close()

print(f"🔍 Successfully opened Connection ID (SPID): {spid} to the Fabic Warehouse.")

🔍 Successfully opened Connection ID (SPID): 54 to the Fabic Warehouse.


In [35]:
#
# Get Foreign Key Relationships from the Fabric Warehouse
#

cursor = conn.cursor()

cursor.execute("""
WITH RelationshipCounts AS (
    SELECT 
        fk.name AS FK_Name, fk_tab.name AS FK_Table, c1.name AS FK_Column, pk.name AS PK_Table, c2.name AS PK_Column,
        CASE 
            -- One-to-One: FK column has a unique index AND PK column has a unique index
            WHEN fk_is_unique.index_id IS NOT NULL AND pk_is_unique.index_id IS NOT NULL THEN 'One-to-One'
            -- Many-to-One: FK is NOT unique, but PK has a unique index
            WHEN pk_is_unique.index_id IS NOT NULL THEN 'Many-to-One'
            -- Many-to-Many: Neither FK nor PK have unique constraints
            ELSE 'Many-to-Many'
        END AS Cardinality,
        COUNT(*) OVER (PARTITION BY pk.name, c2.name) AS PK_Reference_Count
    FROM sys.foreign_keys AS fk
    JOIN sys.foreign_key_columns AS fkc ON fk.object_id = fkc.constraint_object_id
    JOIN sys.tables AS fk_tab ON fk_tab.object_id = fkc.parent_object_id
    JOIN sys.columns AS c1 ON fkc.parent_column_id = c1.column_id 
        AND fkc.parent_object_id = c1.object_id
    JOIN sys.tables AS pk ON pk.object_id = fkc.referenced_object_id
    JOIN sys.columns AS c2 ON fkc.referenced_column_id = c2.column_id 
        AND fkc.referenced_object_id = c2.object_id
    -- Check if the referenced (PK) column is unique
    LEFT JOIN sys.index_columns AS pk_index_cols ON c2.object_id = pk_index_cols.object_id 
        AND c2.column_id = pk_index_cols.column_id
    LEFT JOIN sys.indexes AS pk_is_unique ON pk_index_cols.object_id = pk_is_unique.object_id 
        AND pk_index_cols.index_id = pk_is_unique.index_id 
        AND pk_is_unique.is_unique = 1  
        AND pk_is_unique.type IN (1, 2) -- Clustered or Non-clustered Unique Index
    -- Check if the foreign key (FK) column is unique
    LEFT JOIN sys.index_columns AS fk_index_cols ON c1.object_id = fk_index_cols.object_id 
        AND c1.column_id = fk_index_cols.column_id
    LEFT JOIN sys.indexes AS fk_is_unique ON fk_index_cols.object_id = fk_is_unique.object_id 
        AND fk_index_cols.index_id = fk_is_unique.index_id 
        AND fk_is_unique.is_unique = 1  
        AND fk_is_unique.type IN (1, 2) -- Clustered or Non-clustered Unique Index
)
SELECT 
    FK_Name, FK_Table, FK_Column, PK_Table, PK_Column, Cardinality
FROM RelationshipCounts
WHERE Cardinality = 'Many-to-One' 
AND PK_Reference_Count > 1    
ORDER BY FK_Name, PK_Table, PK_Column;
""")

# Fetch the results
relationships = cursor.fetchall()

print(f"✅ Successfully extracted {len(relationships):,} forgein key relationships from the database '{DATABASE}'.")

✅ Successfully extracted 45 forgein key relationships from the database 'AdventureWorks_Warehouse'.


In [39]:
#
# Convert the results into TMDL format
#

tmdl_output = ""

previous_fk_name = None
fk_name_counter = 1

for row in relationships:
    fk_name = row.FK_Name
    from_table = row.FK_Table
    from_column = row.FK_Column
    to_table = row.PK_Table
    to_column = row.PK_Column
    cardinality = row.Cardinality

    # Check if the current fk_name is the same as the previous one
    if fk_name == previous_fk_name:
        fk_name_counter += 1
        fk_name = f"{fk_name}_{fk_name_counter}"
    else:
        fk_name_counter = 1  # Reset counter for new FK_Name

    previous_fk_name = row.FK_Name  # Store the original FK name for comparison

    # Convert SQL cardinality description to TMDL format
    if cardinality == "One-to-One":
        from_cardinality = "one"
        to_cardinality = "one"
    elif cardinality == "Many-to-One":
        from_cardinality = "many"
        to_cardinality = "one"
    else:
        from_cardinality = "many"
        to_cardinality = "many"

    # Format as TMDL relationship
    tmdl_output += textwrap.dedent(f"""
        relationship {fk_name}
            crossFilteringBehavior: oneDirection
            fromColumn: {from_table}.{from_column}
            toColumn: {to_table}.{to_column}
            fromCardinality: {from_cardinality}
            toCardinality: {to_cardinality}
    """)

# Display the results
print(f"✅ Successfully created TMDL output:\n{(tmdl_output)}")

✅ Successfully created TMDL output:

relationship FK_DimAccount_DimAccount
    crossFilteringBehavior: oneDirection
    fromColumn: DimAccount.ParentAccountKey
    toColumn: DimAccount.AccountKey
    fromCardinality: many
    toCardinality: one

relationship FK_DimCustomer_DimGeography
    crossFilteringBehavior: oneDirection
    fromColumn: DimCustomer.GeographyKey
    toColumn: DimGeography.GeographyKey
    fromCardinality: many
    toCardinality: one

relationship FK_DimDepartmentGroup_DimDepartmentGroup
    crossFilteringBehavior: oneDirection
    fromColumn: DimDepartmentGroup.ParentDepartmentGroupKey
    toColumn: DimDepartmentGroup.DepartmentGroupKey
    fromCardinality: many
    toCardinality: one

relationship FK_DimEmployee_DimEmployee
    crossFilteringBehavior: oneDirection
    fromColumn: DimEmployee.ParentEmployeeKey
    toColumn: DimEmployee.EmployeeKey
    fromCardinality: many
    toCardinality: one

relationship FK_DimEmployee_DimSalesTerritory
    crossFilteringBehav

In [33]:
#
# Close the SQL connection
#

conn.close()
print("✅ Successfully closed the SQL connection to the Fabic Warehouse.")

✅ Successfully closed the SQL connection to the Fabic Warehouse.
